# Assignment: Global average budgets in the CESM pre-industrial control simulation

## Learning goals

Students completing this assignment will gain the following skills and concepts:

- Continued practice working with the Jupyter notebook
- Familiarity with atmospheric output from the CESM simulation
- More complete comparison of the global energy budget in the CESM control simulation to the observations
- Validation of the annual cycle of surface temperature against observations
- Opportunity to formulate a hypothesis about these global temperature variations
- Python programming skills: basic xarray usage: opening gridded dataset and taking averages

## Instructions

- In a local copy of this notebook (on the JupyterHub or your own device) **add your answers in additional cells**.
- **Complete the required problems** below. 
- Remember to set your cell types to `Markdown` for text, and `Code` for Python code!
- **Include comments** in your code to explain your method as necessary.
- Remember to actually answer the questions. **Written answers are required** (not just code and figures!)
- Submit your solutions in **a single Jupyter notebook** that contains your text, your code, and your figures.
- *Make sure that your notebook* ***runs cleanly without errors:***
    - Save your notebook
    - From the `Kernel` menu, select `Restart & Run All`
    - Did the notebook run from start to finish without error and produce the expected output?
    - If yes, save again and submit your notebook file
    - If no, fix the errors and try again.

## Problem 1: The global energy budget in the CESM control simulation

Compute the **global, time average** of each of the following quantities, and compare them to the observed values from the Trenberth and Fasullo (2012) figure in the course notes:

- Solar Radiation budget:
    - Incoming Solar Radiation, or Insolation
    - Reflected Solar Radiation at the top of atmosphere
    - Solar Radiation Reflected by Surface
    - Solar Radiation Absorbed by Surface
    - Solar Radiation Refelected by Clouds and Atmosphere *(you can calculate this as the difference between the reflected radiation at the top of atmosphere and reflected radiation at the surface)*
    - Total Absorbed Solar Radiation (ASR) at the top of atmosphere
    - Solar Radiation Absorbed by Atmosphere *(you can calculate this as the residual of your budget, i.e. what's left over after accounting for all other absorption and reflection)*
- Longwave Radiation budget:
    - Outgoing Longwave Radiation
    - Upward emission from the surface
    - Downwelling radiation at the surface
- Other surface fluxes:
    - "Thermals", or *sensible heat flux*. *You will find this in the field called `SHFLX` in your dataset.*
    - "Evapotranspiration", or *latent heat flux*. *You will find this in the field called `LHFLX` in your dataset.*
    
*Note we will look more carefully at atmospheric absorption and emission processes later. You do not need to try to calculate terms such as "Emitted by Atmosphere" or "Atmospheric Window"*

**Based on your results above, answer the following questions:**

- Is the CESM control simulation at (or near) **energy balance**? 
- Do you think this simulation is near equilibrium?
- Summarize in your own words what you think are the most important similarities and differences of the global energy budgets in the CESM simulation and the observations.

## Problem 2: Verifying the annual cycle in global mean surface temperature against observations

In the class notes we plotted the **timeseries of global mean surface temperature** in the CESM control simulation, and found an **annual cycle**. The purpose of this exercise is to verify that this phenomenon is also found in the observed temperature record. If so, then we can conclude that it is a real feature of Earth's climate and not an artifact of the numerical model.

For observations, we will use the **NCEP Reanalysis data**.

*Reanalysis data is really a blend of observations and output from numerical weather prediction models. It represents our “best guess” at conditions over the whole globe, including regions where observations are very sparse.*

The necessary data are all served up over the internet. We will look at monthly climatologies averaged over the 30 year period 1981 - 2010.

You can browse the available data here: 
https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis.derived.html

**Surface air temperature** is contained in a file called `air.2m.mon.ltm.nc`, which is found in the collection called `Surface Fluxes`. 

Here's a link directly to the catalog page for this data file:
https://www.esrl.noaa.gov/psd/thredds/catalog/Datasets/ncep.reanalysis.derived/surface_gauss/catalog.html?dataset=Datasets/ncep.reanalysis.derived/surface_gauss/air.2m.day.ltm.nc

Now click on the `OPeNDAP` link. A page opens up with lots of information about the contents of the file. The `Data URL` is what we need to read the data into our Python session. For example, this code opens the file and displays a list of the variables it contains:

In [1]:
import xarray as xr
url = "http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/surface_gauss/air.2m.mon.ltm.nc"
ncep_air2m = xr.open_dataset(url)

ncep_air2m

/Users/nfeldl/opt/anaconda3/envs/climlab/lib/python3.10/site-packages/xarray/coding/times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/nfeldl/opt/anaconda3/envs/climlab/lib/python3.10/site-packages/xarray/coding/times.py:144: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/Users/nfeldl/opt/anaconda3/envs/climlab/lib/python3.10/site-packages/xarray/core/indexing.py:423: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, contin

<xarray.Dataset>
Dimensions:             (lon: 192, time: 12, lat: 94, nbnds: 2)
Coordinates:
  * lon                 (lon) float32 0.0 1.875 3.75 5.625 ... 354.4 356.2 358.1
  * time                (time) object 0001-01-01 00:00:00 ... 0001-12-01 00:0...
  * lat                 (lat) float32 88.54 86.65 84.75 ... -84.75 -86.65 -88.54
Dimensions without coordinates: nbnds
Data variables:
    climatology_bounds  (time, nbnds) datetime64[ns] ...
    air                 (time, lat, lon) float32 ...
    valid_yr_count      (time, lat, lon) float32 ...
Attributes:
    Conventions:                    COARDS
    title:                          mean daily NMC reanalysis (1958)
    description:                    Data is from NMC initialized reanalysis\n...
    platform:                       Model
    not_missing_threshold_percent:  minimum 3% values input to have non-missi...
    history:                        Created 2011/07/12 by doMonthLTM\nConvert...
    dataset_title:                  NCEP-NCAR Reanalysis 1
    References:                     http://www.psl.noaa.gov/data/gridded/data...

The temperature data is called `air`. Take a look at the details:

In [2]:
ncep_air2m.air

<xarray.DataArray 'air' (time: 12, lat: 94, lon: 192)>
[216576 values with dtype=float32]
Coordinates:
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * time     (time) object 0001-01-01 00:00:00 ... 0001-12-01 00:00:00
  * lat      (lat) float32 88.54 86.65 84.75 82.85 ... -84.75 -86.65 -88.54
Attributes:
    long_name:     Monthly Long Term Mean of Air Temperature
    valid_range:   [150. 400.]
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    level_desc:    2 m
    statistic:     Long Term Mean
    parent_stat:   Mean
    actual_range:  [198.33992 311.8952 ]
    dataset:       NCEP Reanalysis Derived Products
    _ChunkSizes:   [  1  94 192]

Notice that the dimensions are `(time: 12, lat: 94, lon: 192)`. The time dimension is calendar months. But note that the lat/lon grid is not the same as our model output! 

*Think about how you will handle calculating the global average of these data.*

### Your task:

- Make a well-labeled timeseries graph of the global-averaged observed average surface air temperature climatology. 
- Verify that the annual cycle we found in the CESM simulation also exists in the observations.
- In your own words, suggest a plausible physical explanation for why this annual cycle exists. 